# Scraping Tweets from a public Twitter account between two dates

snscraper has a very easy to use python wrapper than can be used to scrape tweets from a public Twitter account.

In [44]:
from tqdm import tqdm
import snscrape.modules.twitter as sntwitter
import pandas as pd
from datetime import date, timedelta

In [54]:
def scrape_twitter(query=None, user=None, since=str(date.today()-timedelta(days=1)), until=str(date.today()+timedelta(days=1))):
    """
    Scrape tweets from Twitter.
    query - search for tweets containing this string
    user - search for tweets by this user
    since - search for tweets since this date
    until - search for tweets until this date (non-inclusive)

    data saved in csv file named "tweets_{user}.csv"
    """

    if all(v is None for v in {query, user}):
        raise ValueError("Must specify either query or user")

    search_str = f"{query} " if query else ""
    search_str += f"from:{user} " if user else ""
    search_str += f"since:{since} " if since else ""
    search_str += f"until:{until} " if until else ""
    print(search_str)
    
    tweets_df = pd.DataFrame(columns=["tweet_id", "date", "user_name", "tweet", "retweets", "likes"])
    tweets_df.to_csv(f"tweets_{query if query else user}.csv", index=False)

    for tweet in tqdm(sntwitter.TwitterSearchScraper(search_str).get_items()):
        tweet_elem = {"tweet_id": tweet.id,
                      "date":tweet.date,
                      "user_name": tweet.user.username,
                      "tweet":tweet.content,
                      "retweets": tweet.retweetCount,
                      "likes": tweet.likeCount
                      }
         # append tweet to dataframe of tweets
        tweet_df = tweets_df.append(tweet_elem, ignore_index = True)
        # continously append new data to the csv
        tweet_df.to_csv(f"tweets_{query if query else user}.csv", mode="a", index=False, header=False)
    
    return

In [ ]:
scrape_twitter(query="converge")

In [42]:
df = pd.read_csv("tweets_ABSCBNNews.csv")
df

,tweet_id,date,user_name,tweet,retweets,likes
0,1531785093923999744,2022-05-31 23:50:00+00:00,ABSCBNNews,Pulis tiklo sa panghahalay umano ng binata sa ...,3,21
1,1531782578042114048,2022-05-31 23:40:00+00:00,ABSCBNNews,Mag-asawa patay matapos pukpukin sa ulo umano ...,2,9
2,1531780062617276417,2022-05-31 23:30:00+00:00,ABSCBNNews,Babae patay matapos sakalin gamit ang cellphon...,1,9
3,1531777544449572865,2022-05-31 23:20:00+00:00,ABSCBNNews,Pulis pinagbabaril ng riding-in-tandem sa Dava...,1,0
4,1531775027837693952,2022-05-31 23:10:00+00:00,ABSCBNNews,Patay ang isang 2-anyos na batang lalaki haban...,1,5
...,...,...,...,...,...,...
233225,1178826862572621825,2019-10-01 00:20:00+00:00,ABSCBNNews,DOH magsasagawa ng 3 round ng sabayang patak k...,1,5
233226,1178825604277428224,2019-10-01 00:15:00+00:00,ABSCBNNews,"""...sobrang nasasaktan din siya kasi the very ...",20,109
233227,1178824346128670722,2019-10-01 00:10:00+00:00,ABSCBNNews,Smuggled na yosi nasabat sa Zamboanga City htt...,4,6
233228,1178823087816527872,2019-10-01 00:05:00+00:00,ABSCBNNews,Pabirong inamin ng Kapamilya star na hindi siy...,2,86
